In [1]:
import pandas as pd
import numpy as np
import collections
from collections import Counter
from collections import OrderedDict
import csv

In [30]:
import sys
file_name = sys.argv[1]+".csv"
print("Input file name = ",file_name)
test_data = pd.read_csv(file_name,header=None)

test_data =  -f


In [3]:
data = pd.read_csv("training-s118.csv",header=None)

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,10
0,C3,34,20,14.463463,8.753004,-143.048132,32.48045,10.0,0.0,9,81.981331
1,C3,34,25,13.383463,4.753004,-156.671764,42.98045,6.0,0.0,8,99.481331
2,C4,31,19,15.263463,3.753004,-122.612685,17.48045,8.0,2.0,4,71.481331
3,C4,33,19,13.233463,-2.746996,-98.771329,14.48045,4.0,9.0,9,55.981331
4,C3,31,23,15.653463,8.003004,-156.671764,37.48045,6.0,1.0,7,93.981331
...,...,...,...,...,...,...,...,...,...,...,...
406,C5,37,22,12.043463,3.253004,-57.900434,22.48045,1.0,5.0,9,42.481331
407,C3,36,13,14.583463,11.003004,-124.315639,35.48045,4.0,4.0,9,96.981331
408,C3,34,11,15.753463,5.253004,-126.018593,38.48045,10.0,0.0,8,109.981331
409,C2,36,18,15.583463,8.503004,-143.048132,38.48045,10.0,10.0,9,125.481331


In [5]:
number_of_class = len(pd.unique(data[0]))

In [6]:
classes = list(pd.unique(data[0]))
classes.sort()

In [7]:
X_train = np.array(data.iloc[: , 1:])

In [8]:
Y_train = np.array(data.iloc[:,0])

### Decision Tree Structure(Monothetic Tree,Information Gain as Mesure)

In [9]:
class Node:
    def __init__(self,depth = 0,stump=(0,0),parent = None):
        self.depth = depth;
        self.parent = parent;
        self.stump = stump;
        self.left = None;
        self.right = None;
        self.isLeaf = True;
        self.label = 0;
        self.T = None;
    
    def predict(self,data):
        if self.isLeaf:
            for item in classes:
                if self.label==item:
                    return item
        else:
            if data[self.stump[0]]>self.stump[1]:
                return self.right.predict(data)
            else:
                return self.left.predict(data)
    def getEntropy(self, frequency_list):
        entropy = 0
        total = sum(frequency_list)
        proportion_list = [number/total for number in frequency_list]
        for i in range(len(proportion_list)):
            p = proportion_list[i]
            if p>0:
                entropy = entropy - p*np.log(p)
            else:
                entropy = entropy - 0
                
        return entropy;
    
    def getStump(self, X, y):
        
        class_frequency = dict(collections.Counter(y))
        class_frequency = dict(OrderedDict(sorted(class_frequency.items())))

        n = y.size;
        bestObjective = float("-inf");
        
        for i in range(X.shape[1]):
            if self.parent is not None and i == self.parent.stump[0]:
                continue;
            candidateThresholds = np.sort(X[:,i]);
            idx = np.argsort(X[:, i]);
            for j in range(1,candidateThresholds.size-1):
                S1 = y[idx[:j]];
                S2 = y[idx[j:]];
                
                S1_class_frequency = dict(collections.Counter(S1))
                S1_class_frequency = dict(OrderedDict(sorted(S1_class_frequency.items())))
                
                S2_class_frequency = dict(collections.Counter(S2))
                S2_class_frequency = dict(OrderedDict(sorted(S2_class_frequency.items())))
                
                info = (S1.size/n)*self.getEntropy(list(S1_class_frequency.values()))+(S2.size/n)*self.getEntropy(list(S2_class_frequency.values()))
                
                candidateObjective = self.getEntropy(list(class_frequency.values()))-info;
                
                if candidateObjective>bestObjective:
                    bestObjective = candidateObjective
                    bestFeat = i
                    bestThresh = candidateThresholds[j]

        return (bestFeat, bestThresh)
    
    def train(self, X ,y ,maxLeafSize, maxDepth):
        class_frequency = dict(collections.Counter(y))
        class_frequency = dict(OrderedDict(sorted(class_frequency.items())))
        total_count = sum(class_frequency.values())
        assign_class = None
        status = 0
        for items in class_frequency.items():
            key = items[0]
            value = items[1]
            if value/total_count>0.8:  ### 80 percent Node purity is the pruning criteria
                assign_class = key
                status = 1
            
        
        if y.size<maxLeafSize or self.depth>=maxDepth or status==1:  ### Node size and Depth is also used as prining criteria
            
            if status==0:
                assign_class = max(class_frequency, key=class_frequency.get)
            
            self.isLeaf = True
            self.label = assign_class;
        else:
            self.isLeaf = False
            self.stump = self.getStump(X,y);
            self.left = Node(depth = self.depth+1,parent = self)
            self.right = Node(depth = self.depth+1,parent = self)
            discriminant = X[:, self.stump[0]] - self.stump[1]
            self.left.train( X[discriminant <= 0, :], y[discriminant <= 0], maxLeafSize, maxDepth )
            self.right.train( X[discriminant > 0, :], y[discriminant > 0], maxLeafSize, maxDepth )


### Define Tree

In [10]:
class Tree:
    def __init__( self, maxLeafSize = 0, maxDepth = 0 ):
        self.root = Node()
        self.maxLeafSize = maxLeafSize
        self.maxDepth = maxDepth 
        
    def predict( self, data ):
        return self.root.predict( np.array(data) )
    
    def train( self, X, y ):
        self.root.train( X, y, self.maxLeafSize, self.maxDepth )

In [11]:
DT = Tree(maxLeafSize = 10, maxDepth = 20)

In [12]:
DT.train( X_train, Y_train )

In [13]:
def predictBatch(Data):
    prediction = [];
    for i in range(Data.shape[0]):
        x  = Data[i,:]
        y_hat = DT.predict(x);
        prediction.append(y_hat);
    return prediction

In [14]:
def accuracy(X_test,y_test):
    y_predict = predictBatch(X_test);
    y_predict = np.array(y_predict);
    y_test = np.array(y_test);
    z = y_predict==y_test
    count = np.sum(z)
    accu = count/y_test.shape[0]
    return accu

In [15]:
print("Train accuracy = ",accuracy(X_train,Y_train))

Train accuracy =  0.851581508515815


### Code for Test Data

In [25]:
X_test = np.array(test_data.iloc[: , 1:])

In [26]:
z = predictBatch(X_test)

In [27]:
textfile = open("classifier-s118.txt", "w")
for element in z:
    textfile.write(element + "\n")
textfile.close()

In [29]:
a_file = open("classifier-s118.csv", "w",newline="")
writer = csv.writer(a_file)
for value in z:
    writer.writerow([value])

a_file.close()